In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import  ElasticNet
from sklearn.linear_model import  Ridge
from sklearn.model_selection import cross_val_score
import  statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import  Lasso
from sklearn.model_selection import GridSearchCV
import matplotlib
from sklearn.decomposition import  PCA
from sklearn.cluster import  KMeans
import sys
from src.utils import local_dist, kernel_funcs, alt, calculate_dependent
# sys.path.insert(0,'..')

import numpy as np
import pickle
import time
import src.utils as utils
print("import done")

import done


In [2]:
dataset = "kingHousePrices"
path = "../../Data/" + dataset + "/"
with open(path + 'training_idx.data', 'rb') as filehandle:
    training_idx = pickle.load(filehandle)
with open(path + 'validation_idx.data', 'rb') as filehandle:
    validation_idx = pickle.load(filehandle)
with open(path + 'test_idx.data', 'rb') as filehandle:
    test_idx = pickle.load(filehandle)
with open(path + 'x.data', 'rb') as filehandle:
    x = pickle.load(filehandle)
with open(path + 'y.data', 'rb') as filehandle:
    y = pickle.load(filehandle)
with open(path + 'coords.data', 'rb') as filehandle:
    coords = pickle.load(filehandle)
    
x = x[:, 1:]
X_training, X_validation, X_test = x[training_idx, :], x[validation_idx], x[test_idx, :]
y_training, y_validation, y_test = y[training_idx, :], y[validation_idx], y[test_idx, :]
coords_training, coords_validation, coords_test = coords[training_idx], coords[validation_idx], coords[test_idx]

print("done reading")

done reading


In [3]:
def _build_wi(bandwidths, data, train_coords):
    weights = np.zeros((len(bandwidths), len(train_coords)))
    for k in range(len(bandwidths)):
        pilot = int(bandwidths[k])
        dist = local_dist(data, train_coords, False).reshape(-1)
        bw = np.partition(dist, pilot - 1)[pilot - 1] * 1.0000001
        bw = max(bw, 1.0000001)
        for i in range(len(train_coords)):
                weights[k][i] = kernel_funcs(dist[i] / bw, 'gaussian')
    weights = np.transpose(weights)
    return weights

In [4]:
D_train = np.concatenate((X_training, coords_training), axis = 1)
D_validation = np.concatenate((X_validation, coords_validation), axis = 1)

D_test = np.concatenate((X_test, coords_test), axis = 1)
rg = RandomForestRegressor(n_estimators=60)
rg.fit(D_train, y_training.reshape(-1, ))
print(utils.R2(y_test.reshape(-1).tolist(), rg.predict(D_test)))

0.3709959498341391


In [6]:
print(D_train.shape)

(640, 10)


In [9]:
bandwidths = [250.,   250,   250,  250,   250, 250, 250, 250]
predictions = []
for i in range(len(y_test)):
    wi = _build_wi(bandwidths, coords_test[i], coords_training)
    D_train = wi * X_training
    # print(alt(wi, [1, 1, 1, 1, 1], 'mean').shape)
    b = alt(wi, [1, 1, 1, 1, 1, 1, 1, 1], 'mean') * y_training
    rg = RandomForestRegressor(n_estimators=60)
    rg.fit(D_train, b.reshape(-1, ))
    # print(X_test[i])
    # print(rg.predict(X_test[i].reshape(1, -1)))
    if i%36 == 0:
        print(i/len(y_test)*100)
    predictions.append(rg.predict(X_test[i].reshape(1, -1)))

print(utils.R2(y_test.reshape(-1).tolist(), np.asarray(predictions).reshape(-1).tolist()))

0.0
18.0
36.0
54.0
72.0
90.0
0.38734725968864764
